In [1]:
import pandas as pd
import numpy as np
import sqlite3
from urllib.parse import urlparse
from collections import defaultdict
# from adblockparser import AdblockRules

In [2]:
def get_base_script_url(script_url):
    script_url_no_param = script_url.split("?")[0].split("&")[0].split("#")[0]
    return script_url_no_param.split("://")[-1].strip()

def get_site(site_url):
    return site_url.split(".")[1]


CANVAS_READ_FUNCS = [
    "HTMLCanvasElement.toDataURL",
    "CanvasRenderingContext2D.getImageData"
    ]

CANVAS_WRITE_FUNCS = [
    "CanvasRenderingContext2D.fillText",
    "CanvasRenderingContext2D.strokeText"
    ]

CANVAS_FP_DO_NOT_CALL_LIST = ["CanvasRenderingContext2D.save",
                              "CanvasRenderingContext2D.restore",
                              "HTMLCanvasElement.addEventListener"]

WEBRTC_FP_CALLS = ["RTCPeerConnection.createDataChannel",
                   "RTCPeerConnection.createOffer"]

MIN_CANVAS_TEXT_LEN = 10
COOKIE_READ_IN_JS = "window.document.cookie"

In [3]:
query = """SELECT sv.site_url, sv.visit_id,
        js.script_url, js.operation, js.arguments, js.symbol, js.value
        FROM javascript as js LEFT JOIN site_visits as sv
        ON sv.visit_id = js.visit_id WHERE
        js.script_url <> ''
        """

In [5]:
exp_path = "exp-data/nyt-t1.sqlite"
conn = sqlite3.connect(exp_path)
js = pd.read_sql_query(query, conn)

In [6]:
def extract_host_from_url(url_ls):
    return list(map(lambda x: urlparse(x).netloc.split('.')[1] if len(urlparse(x).netloc.split('.')) > 1 else x, url_ls))

In [7]:
js["first_party_host"] = list(map(lambda x: get_site(x), js["site_url"]))

In [8]:
js["script_host"] = extract_host_from_url(js.script_url)

In [9]:
def is_third_party(primary_host, secondary_host):
    result = []
    for ph, sh in zip(primary_host, secondary_host):
        if ph == sh:
            result.append(False)
        else:
            result.append(True)
    return result

In [10]:
js['is_third_party'] = is_third_party(js.first_party_host, js.script_host)

In [62]:
def read_ab_rules_from_file(filename):
    filter_list = set()
    for l in open(filename):
        if len(l) == 0 or l[0] == '!':  # ignore these lines
            continue
        else:
            filter_list.add(l.strip())
    return filter_list

In [11]:
IMAGE_MIN_HEIGHT = 16
IMAGE_MIN_WIDTH = 16
def safe_int(string):
    """Convert string to int, return 0 if conversion fails."""
    try:
        return int(string)
    except ValueError:
        return 0

def image_is_too_small(arguments):
    if arguments is None:
        return True
    if safe_int(arguments[2]) < IMAGE_MIN_WIDTH or safe_int(arguments[3]) < IMAGE_MIN_HEIGHT:
        return True
    return False

def split_cookie_value(value):
    return set(map(lambda x: (x.split('=')[0].strip(), x.split('=')[1].strip()) if len(x.split('=')) > 1 else (x, None), value.split(';')))

In [12]:
third_party_js_script = js[js['is_third_party'] == True]

In [13]:
third_party_js_script_cookie_operations = third_party_js_script[third_party_js_script['symbol'].str.contains("window.document.cookie")]

In [14]:
third_party_set_cookie = defaultdict(set) # origin_host: (script_host, script_url, value)
third_party_get_cookie = defaultdict(set)

In [15]:
insignificant_cookie = 'Test|testWrite|test|lang|SameSite|Samesite|sameSite|samesite|expires|Path|path|domain|Expires|max-age'
insignificant_cookie_ls = insignificant_cookie.split('|')
for index, row in third_party_js_script_cookie_operations.iterrows():
    if row['value'] is None:
        continue
    if row["operation"] == "set":
        for cookie_name, cookie_value in split_cookie_value(row["value"]):
            if cookie_value == None or cookie_name in insignificant_cookie_ls:
                continue
            third_party_set_cookie[row['first_party_host']].add((row['script_host'], row['script_url'], cookie_name, cookie_value))
    elif row["operation"] == "get":
        for cookie_name, cookie_value in split_cookie_value(row["value"]):
            if cookie_value == None or cookie_name in insignificant_cookie_ls:
                continue
            third_party_get_cookie[row['first_party_host']].add((row['script_host'], row['script_url'], cookie_name, cookie_value))   

In [16]:
def make_cookie_into_df(cookie_dict):
    first_party_ls = []
    script_host_ls = []
    script_url_ls = []
    cookie_name_ls = []
    cookie_value_ls = []
    for first_party, value_set in cookie_dict.items():
        first_party_ls.extend([first_party] * len(value_set))
        for script_host, script_url, cookie_name, cookie_value in value_set:
            script_host_ls.append(script_host)
            script_url_ls.append(script_url)
            cookie_name_ls.append(cookie_name)
            cookie_value_ls.append(cookie_value)
    return pd.DataFrame.from_dict({'origin_site':first_party_ls, 'script_host':script_host_ls, 'script_url': script_url_ls,\
                                   'cookie_name': cookie_name_ls, 'cookie_value': cookie_value_ls})


In [17]:
get_cookie_js = make_cookie_into_df(third_party_get_cookie)
set_cookie_js = make_cookie_into_df(third_party_set_cookie)

In [18]:
def drop_cookie_from_df(df):
    df.drop(df[df.script_host.apply(lambda x: len(x) < 5 and x != 'g')].index, \
                   inplace=True)
    df.drop(df[df.cookie_value.apply(lambda x: len(x) < 5)].index, inplace=True)

In [19]:
drop_cookie_from_df(get_cookie_js)

In [20]:
drop_cookie_from_df(set_cookie_js)

In [21]:
# canvas features
# sample check isin: df_new[df_new['l_ext'].isin([31, 22, 30, 25, 64])]
# third_party_js_image = third_party_js_script[third_party_js_script['symbol'].str.contains("CanvasRenderingContext2D.getImageData")

In [22]:
# third_party_img_trackers = defaultdict(set)
# for index, row in third_party_js_image.iterrows():
#     if image_is_too_small(row['arguments']):
#         third_party_img_trackers[row['first_party_host']].add((row['script_host'], row['script_url']))

NameError: name 'third_party_js_image' is not defined

In [52]:
third_party_img_trackers

defaultdict(set,
            {'mayoclinic': {('medtargetsystem',
               'https://www.medtargetsystem.com/javascript/fp.js?5')}})

In [23]:
def get_base_script_url(script_url):
    script_url_no_param = script_url.split("?")[0].split("&")[0].split("#")[0]
    return script_url_no_param.split("://")[-1].strip()

In [24]:
MIN_FONT_FP_FONT_COUNT = 50
def canvas_font_fingerprinters(canvas_fonts, canvas_measure_text_calls):
    canvas_font_fingerprinting = defaultdict(set)
    for first_party_url, script_dict in canvas_fonts.items():
        for script_base_url, fonts_value in script_dict.items():
            if len(fonts_value) < MIN_FONT_FP_FONT_COUNT:
                continue
            for measure_url, args, call_count in canvas_measure_text_calls[first_party_url].items():
                if call_count > MIN_FONT_FP_FONT_COUNT and measure_url == script_base_url:
                    canvas_font_fingerprinting.add((first_party_url, script_base_url))
                    
    return canvas_font_fingerprinting

In [25]:
def convert_text(arguments):
    if not arguments:
        return ""
    return arguments[0].encode('ascii', 'ignore')

In [26]:
canvas_reads = defaultdict(set)
canvas_text = defaultdict(lambda: defaultdict(set))
canvas_write = defaultdict(lambda: defaultdict(set))
canvas_styles = defaultdict(lambda: defaultdict(set))
canvas_banned_calls = defaultdict(set)
canvas_fonts = defaultdict(lambda: defaultdict(set))
canvas_measure_text_calls = defaultdict(lambda: defaultdict(int))
webrtc_calls = defaultdict(lambda: defaultdict(set))
for index, row in third_party_js_script.iterrows():
    script_host = row.script_host
    base_script_url = get_base_script_url(row.script_url)
    if row.symbol in CANVAS_READ_FUNCS and row.operation == "call":
        if row.symbol == "CanvasRenderingContext2D.getImageData" and image_is_too_small(row['arguments']):
            continue
        canvas_reads[row.first_party_host].add((row.script_host, base_script_url))
    elif row.symbol in CANVAS_WRITE_FUNCS:
        text = convert_text(row.arguments)
        if len(text) >= MIN_CANVAS_TEXT_LEN:
            canvas_write[row.first_party_host][base_script_url].add(text)
    elif row.symbol == "CanvasRenderingContext2D.fillStyle" and row.operation == "set":
        if row.value != None:
            canvas_styles[row.first_party_host][base_script_url].add(row.value)
    elif row.operation == "call" and row.symbol in CANVAS_FP_DO_NOT_CALL_LIST:
        canvas_banned_calls[row.first_party_host].add((script_host, base_script_url))
    elif row.symbol == "CanvasRenderingContext2D.font" and row.operation == "set":
        canvas_fonts[row.first_party_host][base_script_url].add(row.value)
    elif row.symbol == "CanvasRenderingContext2D.measureText" and row.operation == "call":
        canvas_measure_text_calls[row.first_party_host][(base_script_url, row.arguments)] += 1
    elif (row.operation == "call" and row.symbol in WEBRTC_FP_CALLS) or\
                (row.operation == "set" and
                 row.symbol == "RTCPeerConnection.onicecandidate"):
        webrtc_calls[row.first_party_host][base_script_url].add(row.symbol)

In [27]:
def get_webrtc_fingerprinters(webrtc_calls):
    webrtc_fingerprinters = defaultdict(set)
    for first_host, script_dict in webrtc_calls.items():
        for script_url, sym_set in script_dict.items():
            if len(sym_set) == 3:
                webrtc_fingerprinters[first_host].add(script_url)
    return webrtc_fingerprinters

In [28]:
get_webrtc_fingerprinters(webrtc_calls)
canvas_font_fingerprinters(canvas_fonts, canvas_measure_text_calls)

defaultdict(set, {})

In [186]:
set(zip(js.script_url, js.is_third_party))[:5]

TypeError: 'set' object is not subscriptable

In [188]:
set(js.script_url)

191

In [29]:
set_cookie_js

,origin_site,script_host,script_url,cookie_name,cookie_value
0,webmd,liadm,https://b-code.liadm.com/sync-container.js,_liChk,0.14646394239970872
1,webmd,adobedtm,https://assets.adobedtm.com/extensions/EP5e9ec...,s_sq,%5B%5BB%5D%5D
2,webmd,amazonaws,https://s3.amazonaws.com/ki.js/38969/hrI.js,ki_t,1606506096713%3B1606506096713%3B1606506128050%...
5,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...
6,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,mbox,session#008c00a7adb142598621f56b2baa7b16#16065...
7,webmd,adobedtm,https://assets.adobedtm.com/extensions/EP5e9ec...,ctrHealthCenter,%5B%5BB%5D%5D
8,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...
9,webmd,criteo,https://gum.criteo.com/sync?r=2&c=321&j=window...,cto_bundle,keV2al9sVXlBS3VvNTVtYmxkTXMlMkJpbXF0WUpGQ3BTcH...
10,webmd,adobedtm,https://assets.adobedtm.com/extensions/EP5e9ec...,aam,aam%3D999991
12,webmd,criteo,https://gum.criteo.com/sync?r=2&c=321&j=window...,cto_bundle,3dgWBF9sVXlBS3VvNTVtYmxkTXMlMkJpbXF0WU1WbmQ4Y0...


In [30]:
conn_cookies = sqlite3.connect(exp_path)
javascript_cookies = pd.read_sql_query("select visit_id, record_type, change_cause, host, name, value, time_stamp from javascript_cookies;", conn_cookies)

In [31]:
javascript_cookies.host.unique()

array(['.www.webmd.com', '.js.webmd.com', '.img.webmd.com',
       '.css.webmd.com', '.webmd.com', '.media.net', '.demdex.net',
       '.scorecardresearch.com', 'www.webmd.com', '.turn.com',
       '.rlcdn.com', 'ibclick.stream', '.liadm.com', 'i.liadm.com',
       '.adsrvr.org', '.adnxs.com', '.bidswitch.net', '.doubleclick.net',
       '.dotomi.com', '.dpm.demdex.net', '.mathtag.com', '.addthis.com',
       '.amazon-adsystem.com', 'global.ib-ibi.com', '.pubmatic.com',
       '.ads.pubmatic.com', '.adsymptotic.com', '.fiftyt.com',
       '.everesttech.net', '.perf-serving.com', 'ib.mookie1.com',
       '.ib.mookie1.com', '.dnacdn.net', '.1rx.io',
       'eus.rubiconproject.com', '.casalemedia.com', '.yahoo.com',
       '.advertising.com', '.3lift.com', '.yieldmo.com',
       '.contextweb.com', 'bh.contextweb.com', '.rubiconproject.com',
       '.analytics.yahoo.com', 'sync.srv.stackadapt.com',
       '.srv.stackadapt.com', '.sitescout.com', '.adentifi.com',
       '.advangelists.com',

In [32]:
def clean_cookie_host(series):
    
    def helper(x):
        ls = x.strip('.').replace('.uk','').replace('.au','').split('.')
        return ls[-2]
    #     ind_com = ls.index('com')
    #     if ind_com > 0:
    #         return ls[ind_com - 1]
    #     elif ind_com == 0:
    #         return 'unknown'
    #     else:
    #         return 
    return list(map(lambda x: helper(x), series))

In [33]:
javascript_cookies['cleaned_host'] = clean_cookie_host(javascript_cookies.host)

In [34]:
javascript_cookies.host[javascript_cookies.host.str.contains("com.com")]

Series([], Name: host, dtype: object)

In [35]:
pd.merge(javascript_cookies, set_cookie_js, how ='inner',left_on=['name', 'value'], right_on=['cookie_name', 'cookie_value'])

,visit_id,record_type,change_cause,host,name,value,time_stamp,cleaned_host,origin_site,script_host,script_url,cookie_name,cookie_value
0,1,added-or-changed,explicit,.webmd.com,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CvVersion%7C4.5.2,2020-11-27T19:41:30.376Z,webmd,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CvVersion%7C4.5.2
1,1,added-or-changed,explicit,.webmd.com,mbox,session#008c00a7adb142598621f56b2baa7b16#16065...,2020-11-27T19:41:30.415Z,webmd,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,mbox,session#008c00a7adb142598621f56b2baa7b16#16065...
2,1,added-or-changed,explicit,www.webmd.com,mnet_session_depth,1%7C1606506091348,2020-11-27T19:41:31.351Z,webmd,webmd,media,https://contextual.media.net/bidexchange.js?ci...,mnet_session_depth,1%7C1606506091348
3,1,added-or-changed,explicit,.webmd.com,_li_dcdm_c,.webmd.com,2020-11-27T19:41:31.730Z,webmd,webmd,liadm,https://b-code.liadm.com/a-00xm.min.js,_li_dcdm_c,.webmd.com
4,1,added-or-changed,explicit,.webmd.com,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35,2020-11-27T19:41:31.732Z,webmd,webmd,liadm,https://b-code.liadm.com/a-00xm.min.js,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35
5,1,deleted,overwrite,.webmd.com,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35,2020-11-27T19:41:33.226Z,webmd,webmd,liadm,https://b-code.liadm.com/a-00xm.min.js,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35
6,1,added-or-changed,explicit,.webmd.com,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35,2020-11-27T19:41:33.229Z,webmd,webmd,liadm,https://b-code.liadm.com/a-00xm.min.js,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35
7,1,deleted,overwrite,.webmd.com,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35,2020-11-27T19:41:51.380Z,webmd,webmd,liadm,https://b-code.liadm.com/a-00xm.min.js,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35
8,1,added-or-changed,explicit,.webmd.com,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35,2020-11-27T19:41:51.380Z,webmd,webmd,liadm,https://b-code.liadm.com/a-00xm.min.js,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35
9,1,deleted,overwrite,.webmd.com,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35,2020-11-27T19:41:53.741Z,webmd,webmd,liadm,https://b-code.liadm.com/a-00xm.min.js,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35


In [25]:
set_cookies_merged = pd.merge(javascript_cookies, set_cookie_js, how ='inner',left_on=['name', 'value'], right_on=['cookie_name', 'cookie_value'])

In [26]:
js_cookies_distinct = javascript_cookies[['visit_id','host', 'name', 'value', 'cleaned_host']].drop_duplicates()

In [27]:
set_cookies_merged = pd.merge(js_cookies_distinct, set_cookie_js, how ='right',left_on=['name', 'value'], right_on=['cookie_name', 'cookie_value'])

In [28]:
get_cookies_merged = pd.merge(js_cookies_distinct, get_cookie_js, how ='right',left_on=['name', 'value'], right_on=['cookie_name', 'cookie_value'])

In [29]:
set_cookies_merged.shape

(108, 10)

In [30]:
get_cookies_merged.shape

(1748, 10)

In [31]:
pd.concat([set_cookies_merged, get_cookies_merged])

,visit_id,host,name,value,cleaned_host,origin_site,script_host,script_url,cookie_name,cookie_value
0,1.0,.webmd.com,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CvVersion%7C4.5.2,webmd,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CvVersion%7C4.5.2
1,1.0,.webmd.com,mbox,session#008c00a7adb142598621f56b2baa7b16#16065...,webmd,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,mbox,session#008c00a7adb142598621f56b2baa7b16#16065...
2,1.0,www.webmd.com,mnet_session_depth,1%7C1606506091348,webmd,webmd,media,https://contextual.media.net/bidexchange.js?ci...,mnet_session_depth,1%7C1606506091348
3,1.0,.webmd.com,_li_dcdm_c,.webmd.com,webmd,webmd,liadm,https://b-code.liadm.com/a-00xm.min.js,_li_dcdm_c,.webmd.com
4,1.0,.webmd.com,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35,webmd,webmd,liadm,https://b-code.liadm.com/a-00xm.min.js,_lc2_fpi,87ca8f4d258a--01er5kk55nbdhke6vdd9ys5s35
5,1.0,.webmd.com,cto_write_test,testWrite,webmd,webmd,criteo,https://gum.criteo.com/sync?r=2&c=321&j=window...,cto_write_test,testWrite
6,3.0,.nytimes.com,cto_write_test,testWrite,nytimes,webmd,criteo,https://gum.criteo.com/sync?r=2&c=321&j=window...,cto_write_test,testWrite
7,1.0,.webmd.com,cto_write_test,testWrite,webmd,nytimes,criteo,https://gum.criteo.com/sync?r=2&c=321&j=window...,cto_write_test,testWrite
8,3.0,.nytimes.com,cto_write_test,testWrite,nytimes,nytimes,criteo,https://gum.criteo.com/sync?r=2&c=321&j=window...,cto_write_test,testWrite
9,1.0,ibclick.stream,_ibp,0:ki0o8yeo:45b107e5-2209-4665-a24e-435f8af67631,ibclick,webmd,stream,https://ibclick.stream/assets/js/track/dist/js...,_ibp,0:ki0o8yeo:45b107e5-2209-4665-a24e-435f8af67631


In [95]:
set_cookies_merged.sort_values(by=['value'])

,visit_id,host,name,value,cleaned_host,origin_site,script_host,script_url,cookie_name,cookie_value
35,1.0,.webmd.com,s_sq,%5B%5BB%5D%5D,webmd,webmd,adobedtm,https://assets.adobedtm.com/extensions/EP5e9ec...,s_sq,%5B%5BB%5D%5D
20,1.0,.webmd.com,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...,webmd,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...
18,1.0,.webmd.com,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...,webmd,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...
21,1.0,.webmd.com,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...,webmd,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...
17,1.0,.webmd.com,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...,webmd,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...
16,1.0,.webmd.com,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...,webmd,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...
15,1.0,.webmd.com,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...,webmd,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CMCMID%7C45583913...
0,1.0,.webmd.com,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CvVersion%7C4.5.2,webmd,webmd,adobedtm,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,AMCV_16AD4362526701720A490D45%40AdobeOrg,-432600572%7CMCIDTS%7C18594%7CvVersion%7C4.5.2
94,3.0,www.nytimes.com,_chartbeat2,.1606506416769.1606506416769.1.B68Vf2Bu8WN0D4M...,nytimes,nytimes,chartbeat,https://static.chartbeat.com/js/chartbeat.js,_chartbeat2,.1606506416769.1606506416769.1.B68Vf2Bu8WN0D4M...
3,1.0,.webmd.com,_li_dcdm_c,.webmd.com,webmd,webmd,liadm,https://b-code.liadm.com/a-00xm.min.js,_li_dcdm_c,.webmd.com
